In [8]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 16


In [9]:
import numpy as np

class normalTreeNode:
    def __init__(self):
        self.data = None
        self.left = None
        self.right = None
        self.cloest = np.inf
        self.farthest = 0
        self.inner = np.nan
        self.outter = np.nan

class parallelTreeNode:
    def __init__(self):
        self.branches = []
        self.anchors = []
        self.cloest = np.inf #cloest and farthest are the ball with respect with the 0th anchor
        self.farthest = 0


# There are different partition method
# - many branches, go to whichever is the cloest. When same dis, go to the lower index
# - same as normal tree but just pack some more objects in the node 


In [10]:
dataset = [np.random.uniform(size=(2)) for i in range(10000)]
def d(a, b):
    return np.sqrt(np.sum((a-b)**2))

In [13]:
import random
from functools import cmp_to_key
# https://stackoverflow.com/questions/5213033/sort-a-list-of-lists-with-a-custom-compare-function

def buildNormalTree(root, dataset):
    if len(dataset) == 0:
        return None
    if len(dataset) == 1:
        root.data=dataset[0]
        root.inner = 0
        root.farthest = 0
        root.cloest = 0
        root.outter = 0
        return root
    if len(dataset) == 2:
        root.data=dataset[0]
        dis = d(dataset[1], dataset[0])
        root.inner = dis
        root.farthest = dis
        root.cloest = dis
        root.outter = dis + 1 # never  reach
        root.left = buildNormalTree(normalTreeNode(), [dataset[1]])
        return root
    random_index = random.randint(0, len(dataset)-1)
    dataset[random_index], dataset[-1] = dataset[-1], dataset[random_index]
    root.data = dataset[-1]
    dataset = dataset[:-1]
    distances = []
    for index, datapoint in enumerate(dataset): #this can be parrelized, but it is not the point at this point
        dis = d(root.data, datapoint)
        root.cloest = min(root.cloest, dis)
        root.farthest = max(root.farthest, dis)
        distances.append(dis)
    
    distances_ = sorted(distances)
    root.inner = distances_[len(distances_)//2-1]
    root.outter = distances_[len(distances_)//2]
    L = []
    R = []
    for i in range(len(dataset)):
        if distances[i]<=root.inner:
            L.append(dataset[i])
        elif distances[i]>=root.outter:
            R.append(dataset[i])
        else:
            print("OOBA")
    root.left = buildNormalTree(normalTreeNode(),L)
    root.right = buildNormalTree(normalTreeNode(),R)
    return root
buildNormalTree(normalTreeNode(), dataset)


In [ ]:
from numpy.random import default_rng

def buildParallelTree(root, dataset):
    if len(dataset) == 0:
        return None
    if len(dataset) == 1:
        root.anchors=[dataset[0]]
        root.farthest = 0
        root.cloest = 0
        return root
    if len(dataset) == 2:
        root.anchors=[dataset[0], dataset[1]]
        root.inner = 0
        root.farthest = 0
        root.cloest = 0
        root.outter = 0
        return root

    number_of_anchors = min(CORES, len(dataset))
    np.random.choice(dataset, size=number_of_anchors, replace=False) #GPT